In [1]:
import requests
import pandas as pd
import json
from legcop import LegiScan

legiscan_api_key='c8d73b2173b9881d495a8a05d59d860d'
legis = LegiScan(legiscan_api_key)

API Example

In [3]:
state='ca'
ca_sessions = legis.get_session_list(state=state)
print(ca_sessions)
start_most_recent_session = ca_sessions[0]['year_start']

datasetlist = legis.get_dataset_list(state=state,
                                     year=start_most_recent_session)

access_key = datasetlist[0]['access_key']
session_id =  datasetlist[0]['session_id']
dataset = legis.get_dataset(session_id=session_id, access_key=access_key)
assert dataset['status']=='OK'
readable_dataset = legis.recode_zipfile(dataset)
namelist = readable_dataset.namelist()
print(namelist)
item=namelist[0]
content = readable_dataset.read(item)
content = content.decode('UTF-8')
content = json.loads(content)

for key, value in content['bill'].items():
    print('{}: {}'.format(key, value))

[{'session_id': 2016, 'state_id': 5, 'year_start': 2023, 'year_end': 2024, 'prefile': 0, 'sine_die': 0, 'prior': 0, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2023-2024 Regular Session', 'session_name': '2023-2024 Session', 'dataset_hash': '89d69e51e88506408d80690952568b18', 'session_hash': '89d69e51e88506408d80690952568b18', 'name': '2023-2024 Regular Session'}, {'session_id': 1791, 'state_id': 5, 'year_start': 2021, 'year_end': 2022, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2021-2022 Regular Session', 'session_name': '2021-2022 Session', 'dataset_hash': '48ae3222e08e6cd730ef7c818d467561', 'session_hash': '48ae3222e08e6cd730ef7c818d467561', 'name': '2021-2022 Regular Session'}, {'session_id': 1624, 'state_id': 5, 'year_start': 2019, 'year_end': 2020, 'prefile': 0, 'sine_die': 1, 'prior': 1, 'special': 0, 'session_tag': 'Regular Session', 'session_title': '2019-2020 Regular Session', 'session_name'

## CA Bill Master List

In [79]:
# Pull in full master list of CA bills
ca_masterlist = legis.get_master_list(state='ca')
df = pd.json_normalize(ca_masterlist)

In [80]:
# Data cleanup
df = df.drop(['prefile', 'sine_die', 'prior', 'special'], axis=1)
df['session_id'] = df['session_id'].fillna(method='ffill')
df['state_id'] = df['state_id'].fillna(method='ffill')
df['year_start'] = df['year_start'].fillna(method='ffill')
df['year_end'] = df['year_end'].fillna(method='ffill')
df['session_tag'] = df['session_tag'].fillna(method='ffill')
df['session_title'] = df['session_title'].fillna(method='ffill')
df['session_name'] = df['session_name'].fillna(method='ffill')
df['status_date'] = pd.to_datetime(df['status_date'])
df['last_action_date'] = pd.to_datetime(df['last_action_date'])
df = df.drop(index=0)
df = df.sort_values(by='last_action_date', ascending=False).reset_index(drop=True)
df.head(5)

,session_id,state_id,year_start,year_end,session_tag,session_title,session_name,bill_id,number,change_hash,url,status_date,status,last_action_date,last_action,title,description
0,2016.0,5.0,2023.0,2024.0,Regular Session,2023-2024 Regular Session,2023-2024 Session,1714744.0,SB793,689c9ea74ff9a2f5e04f5248f8088150,https://legiscan.com/CA/bill/SB793/2023,2023-05-11,2.0,2023-06-28,From committee: Do pass and re-refer to Com. on APPR. with recommendation: To consent calendar. (Ayes 13. Noes 0.) (June 28). Re-referred to Com. on APPR.,Insurance: privacy notices and personal information.,"An act to add Section 791.045 to the Insurance Code, relating to insurance."
1,2016.0,5.0,2023.0,2024.0,Regular Session,2023-2024 Regular Session,2023-2024 Session,1732852.0,SB884,27884e4d09997f3f05698bc182699ad6,https://legiscan.com/CA/bill/SB884/2023,2023-05-11,2.0,2023-06-28,From committee: Do pass. Ordered to consent calendar. (Ayes 14. Noes 0.) (June 28).,Gambling Control Act.,"An act to amend Section 19805 of the Business and Professions Code, relating to gambling."
2,2016.0,5.0,2023.0,2024.0,Regular Session,2023-2024 Regular Session,2023-2024 Session,1709728.0,AB1028,e4cbe328a8cd1ea06df5ba8f55b64827,https://legiscan.com/CA/bill/AB1028/2023,2023-05-31,2.0,2023-06-28,"From committee chair, with author's amendments: Amend, and re-refer to committee. Read second time, amended, and re-referred to Com. on PUB S.",Reporting of crimes: mandated reporters.,"An act to amend, repeal, and add Sections 11160, 11161, 11163.2, and 11163.3 of the Penal Code, relating to reporting of crimes."
3,2016.0,5.0,2023.0,2024.0,Regular Session,2023-2024 Regular Session,2023-2024 Session,1691365.0,SB275,49e269da745e7342df664e9d8abed896,https://legiscan.com/CA/bill/SB275/2023,2023-06-28,3.0,2023-06-28,In Senate. Ordered to engrossing and enrolling.,State Oil and Gas Supervisor: Senate confirmation.,"An act to amend Section 690 of the Public Resources Code, relating to oil and gas."
4,2016.0,5.0,2023.0,2024.0,Regular Session,2023-2024 Regular Session,2023-2024 Session,1706000.0,SB462,77d04195d8cfdfb30be6b608a955646b,https://legiscan.com/CA/bill/SB462/2023,2023-04-20,2.0,2023-06-28,From committee: Do pass. Ordered to consent calendar. (Ayes 11. Noes 0.) (June 27).,General assistance.,"An act to add Section 17006.1 to the Welfare and Institutions Code, relating to public social services."


Getting all bill titles

In [81]:
#TODO create function to do this
unique_bill_title = df['title'].unique()
for i in range(0, len(unique_bill_title)):
    print(unique_bill_title[i])

Insurance: privacy notices and personal information.
Gambling Control Act.
Reporting of crimes: mandated reporters.
State Oil and Gas Supervisor: Senate confirmation.
General assistance.
Affordable Housing Credit Improvement Act of 2023.
Valley Fever Awareness Month.
Residential real property: foreclosure.
Light pollution control.
County government: contract legal counsel: elected treasurer-tax collector.
Firearms.
State Threat Assessment Center: transnational criminal organizations.
Monterey-Salinas Transit District: public contracting.
Workers’ compensation: utilization review.
Value of care review.
Local government: internet websites and email addresses.
Gun violence restraining orders: body armor.
Student financial aid: immigrants seeking asylum.
Criminal procedure: victims’ rights.
Foster youth.
Medical evidentiary examinations: reimbursement.
Covered California: data sharing.
Public postsecondary education: disabled student services: assessments.
Organic waste: reduction goals: l

In [82]:
# We can possibly categorize the bills by groups if they include ":". This is just an idea off first glance, not all bills follow this format

In [83]:
df['title_group'] = df['title'].str.split(':').str[0]
df['title_group'] = df['title_group'].str.title()
df['title_group'] = df['title_group'].str.replace('.','')
df['title_group'].value_counts()

title_group
Budget Act Of 2023                                                                                                                                                                                                                                                    189
Medi-Cal                                                                                                                                                                                                                                                               36
Vehicles                                                                                                                                                                                                                                                               34
Elections                                                                                                                                                                                     